In [1]:
import time
import numpy as np
import tensorflow as tf
import re
from collections import Counter
from time import sleep
import json
import random

with open('data/pos_train.json') as json_data:
    d = json.load(json_data)

text=[]
l=0

for i in d:
    for j in i:
        for k in j:
            text.append(k)

tt = []
for i in range(len(text)):
    if(text[i]!=''):
        tt.append(text[i])

text = tt

for i in range(len(text)):
    if(text[i]=='$'or text[i]=='PDT' or text[i]=='WP$' or text[i]=="SYM" or text[i]=='LS' or text[i]=='#' or text[i]=='UH'):
        text[i]='OTHER'
    

word_counts = Counter(text)

word_counts.most_common()

[('NN', 391092),
 ('IN', 313428),
 ('NNP', 286327),
 ('DT', 246818),
 ('JJ', 194522),
 ('NNS', 150057),
 (',', 138811),
 ('.', 92716),
 ('CC', 86433),
 ('VBD', 83807),
 ('CD', 77651),
 ('RB', 71899),
 ('VBN', 69596),
 ('TO', 48800),
 ('VBZ', 41862),
 ('VB', 40118),
 ('VBG', 35283),
 (':', 34343),
 ('VBP', 28324),
 ('PRP', 24380),
 ('FW', 22213),
 ('PRP$', 18771),
 ('POS', 14923),
 ('WDT', 13756),
 ("''", 13344),
 ('``', 12561),
 ('MD', 11441),
 ('NNPS', 10637),
 ('JJS', 6400),
 ('JJR', 6373),
 ('WRB', 5119),
 ('WP', 4378),
 ('RP', 3189),
 ('RBR', 2910),
 ('OTHER', 2645),
 ('EX', 2623),
 ('RBS', 2274)]

In [2]:
def create_lookup_tables(words):
    """
    Create lookup tables for vocabulary
    :param words: Input list of words
    :return: A tuple of dicts.  The first dict....
    """
    word_counts = Counter(words)
    sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)
    int_to_vocab = {ii: word for ii, word in enumerate(sorted_vocab)}
    vocab_to_int = {word: ii for ii, word in int_to_vocab.items()}

    return vocab_to_int, int_to_vocab

def get_target(words, idx, window_size=5):
    ''' Get a list of words in a window around an index. '''
    
    R = np.random.randint(1, window_size+1)
    start = idx - R if (idx - R) > 0 else 0
    stop = idx + R
    target_words = set(words[start:idx] + words[idx+1:stop+1])
    
    return list(target_words)


def get_batches(words, batch_size, window_size=5):
    ''' Create a generator of word batches as a tuple (inputs, targets) '''
    
    n_batches = len(words)//batch_size
    
    # only full batches
    words = words[:n_batches*batch_size]
    for idx in range(0, len(words), batch_size):
        x, y = [], []
        batch = words[idx:idx+batch_size]
        for ii in range(len(batch)):
            batch_x = batch[ii]
            batch_y = get_target(batch, ii, window_size)
            y.extend(batch_y)
            x.extend([batch_x]*len(batch_y))
        yield x, y

In [3]:
words = text

print("Total words: {}".format(len(words)))
print("Unique words: {}".format(len(set(words))))

# vocab_to_int, int_to_vocab = utils.create_lookup_tables(words)
vocab_to_int, int_to_vocab = create_lookup_tables(words)
int_words = [vocab_to_int[word] for word in words]

word_counts = Counter(int_words)
total_count = len(int_words)
weights = [1 - 4*count/total_count for word, count in word_counts.items()]
weights = 2*np.multiply(weights, weights)
weights = weights.astype(np.float32)

Total words: 2609824
Unique words: 37


In [4]:
def get_train_word():
    threshold = random.uniform(0.08, 0.1)
    word_counts = Counter(int_words)
    total_count = len(int_words)
    freqs = {word: count/total_count for word, count in word_counts.items()}
    p_drop = {word: 1 - np.sqrt(threshold/freqs[word]) for word in word_counts}
    train_words = [word for word in int_words if random.random() < (1 - p_drop[word])]
    return train_words, threshold

In [5]:
batch_size = 1000
window_size = 5
n_vocab = len(int_to_vocab)
n_embedding =  20
starter_learning_rate = 0.0001
decay_steps = 2000
decay_rate = 0.96
lambda_l2 = 0.05

In [6]:
with tf.device('/device:GPU:3'):

    train_graph = tf.Graph()
    with train_graph.as_default():
        inputs = tf.placeholder(tf.int32, [None], name='inputs')
        labels = tf.placeholder(tf.int32, [None], name='labels')

        embedding = tf.Variable(tf.random_uniform((n_vocab, n_embedding), -1, 1))
        embed = tf.nn.embedding_lookup(embedding, inputs) # use tf.nn.embedding_lookup to get the hidden layer output
        embed_drop = tf.nn.dropout(embed, 0.8)

        softmax_w = tf.Variable(tf.truncated_normal((n_vocab, n_embedding)), name="softmax_w") # create softmax weight matrix here
        softmax_b = tf.Variable(tf.zeros(n_vocab), name="softmax_bias") # create softmax biases here

        logits = tf.matmul(embed_drop, tf.transpose(softmax_w)) + softmax_b
        
        class_weight = tf.constant(weights)
        weighted_logits = logits * class_weight


        
        labels_one_hot = tf.one_hot(labels, n_vocab)
        
        tv_all = tf.trainable_variables()

        l2_loss = lambda_l2 * tf.reduce_sum([ tf.nn.l2_loss(v) for v in tv_all ])

        loss = tf.nn.softmax_cross_entropy_with_logits(labels=labels_one_hot, logits=weighted_logits)
        
        cost = tf.reduce_mean(loss) + l2_loss
        
        global_step = tf.Variable(0, trainable=False, name="global_step")

        learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, decay_steps, decay_rate)


        optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost, global_step=global_step)
        
        norm = tf.sqrt(tf.reduce_sum(tf.square(embedding), 1, keep_dims=True))
        normalized_embedding = embedding / norm

In [7]:
model_name = 'pos_weighted_20'

In [8]:
# If the checkpoints directory doesn't exist:
!mkdir checkpoints/pos_weighted_20

mkdir: cannot create directory ‘checkpoints/pos_weighted_20’: File exists


In [9]:
epochs = 10

In [10]:
with train_graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=train_graph) as sess:
    iteration = 1
    batch_loss = 0
    sess.run(tf.global_variables_initializer())
    
#     saver.restore(sess, tf.train.latest_checkpoint('checkpoints/pos'))
#     embed_mat = sess.run(embedding)
    
    for e in range(1, epochs+1):
        train_words, threshold = get_train_word()
        print("Epoch {}/{}".format(e, epochs), "Threshold: {}".format(threshold), "Length of Training words: {}".format(len(train_words)))
        batches = get_batches(train_words, batch_size, window_size)
        start = time.time()
        for x, y in batches:
            feed = {inputs: x,
                    labels: np.array(y)}
            global_steps, train_loss, _ = sess.run([global_step, cost, optimizer], feed_dict=feed)
            
            batch_loss += train_loss
            
            if iteration % 100== 0: 
                end = time.time()
                print("Global Step: {}".format(global_steps), "Epoch {}/{}".format(e, epochs),
                      "Iteration: {}".format(iteration),
                      "Avg. Training loss: {:.4f}".format(batch_loss/100),
                      "{:.4f} sec/batch".format((end-start)/100))
                batch_loss = 0
                start = time.time()
            
            iteration += 1
    save_path = saver.save(sess, "checkpoints/" + model_name + "/pos.ckpt")
    embed_mat = sess.run(normalized_embedding)

Epoch 1/10 Threshold: 0.08644539633845551 Length of Training words: 2425040
Global Step: 100 Epoch 1/10 Iteration: 100 Avg. Training loss: 30.1618 0.0315 sec/batch
Global Step: 200 Epoch 1/10 Iteration: 200 Avg. Training loss: 29.4988 0.0448 sec/batch
Global Step: 300 Epoch 1/10 Iteration: 300 Avg. Training loss: 28.8032 0.0064 sec/batch
Global Step: 400 Epoch 1/10 Iteration: 400 Avg. Training loss: 28.1691 0.0143 sec/batch
Global Step: 500 Epoch 1/10 Iteration: 500 Avg. Training loss: 27.5454 0.0053 sec/batch
Global Step: 600 Epoch 1/10 Iteration: 600 Avg. Training loss: 26.9275 0.0095 sec/batch
Global Step: 700 Epoch 1/10 Iteration: 700 Avg. Training loss: 26.3362 0.0076 sec/batch
Global Step: 800 Epoch 1/10 Iteration: 800 Avg. Training loss: 25.7386 0.0081 sec/batch
Global Step: 900 Epoch 1/10 Iteration: 900 Avg. Training loss: 25.1564 0.0052 sec/batch
Global Step: 1000 Epoch 1/10 Iteration: 1000 Avg. Training loss: 24.6026 0.0082 sec/batch
Global Step: 1100 Epoch 1/10 Iteration: 11

Global Step: 9000 Epoch 4/10 Iteration: 9000 Avg. Training loss: 6.3155 0.0149 sec/batch
Global Step: 9100 Epoch 4/10 Iteration: 9100 Avg. Training loss: 6.2577 0.0067 sec/batch
Global Step: 9200 Epoch 4/10 Iteration: 9200 Avg. Training loss: 6.1821 0.0067 sec/batch
Global Step: 9300 Epoch 4/10 Iteration: 9300 Avg. Training loss: 6.1144 0.0105 sec/batch
Global Step: 9400 Epoch 4/10 Iteration: 9400 Avg. Training loss: 6.0337 0.0083 sec/batch
Global Step: 9500 Epoch 4/10 Iteration: 9500 Avg. Training loss: 5.9620 0.0072 sec/batch
Global Step: 9600 Epoch 4/10 Iteration: 9600 Avg. Training loss: 5.9120 0.0061 sec/batch
Global Step: 9700 Epoch 4/10 Iteration: 9700 Avg. Training loss: 5.8410 0.0067 sec/batch
Epoch 5/10 Threshold: 0.08843282539255745 Length of Training words: 2437566
Global Step: 9800 Epoch 5/10 Iteration: 9800 Avg. Training loss: 5.7945 0.0066 sec/batch
Global Step: 9900 Epoch 5/10 Iteration: 9900 Avg. Training loss: 5.7403 0.0070 sec/batch
Global Step: 10000 Epoch 5/10 Iter

Global Step: 17700 Epoch 8/10 Iteration: 17700 Avg. Training loss: 3.6585 0.0059 sec/batch
Global Step: 17800 Epoch 8/10 Iteration: 17800 Avg. Training loss: 3.6574 0.0089 sec/batch
Global Step: 17900 Epoch 8/10 Iteration: 17900 Avg. Training loss: 3.6449 0.0061 sec/batch
Global Step: 18000 Epoch 8/10 Iteration: 18000 Avg. Training loss: 3.6318 0.0082 sec/batch
Global Step: 18100 Epoch 8/10 Iteration: 18100 Avg. Training loss: 3.6310 0.0078 sec/batch
Global Step: 18200 Epoch 8/10 Iteration: 18200 Avg. Training loss: 3.6126 0.0096 sec/batch
Global Step: 18300 Epoch 8/10 Iteration: 18300 Avg. Training loss: 3.6083 0.0062 sec/batch
Global Step: 18400 Epoch 8/10 Iteration: 18400 Avg. Training loss: 3.6070 0.0052 sec/batch
Global Step: 18500 Epoch 8/10 Iteration: 18500 Avg. Training loss: 3.5895 0.0057 sec/batch
Global Step: 18600 Epoch 8/10 Iteration: 18600 Avg. Training loss: 3.5938 0.0062 sec/batch
Global Step: 18700 Epoch 8/10 Iteration: 18700 Avg. Training loss: 3.5769 0.0088 sec/batch

In [13]:
with train_graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=train_graph) as sess:
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints/' + model_name))
    embed_mat = sess.run(embedding)

In [11]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

viz_words = n_vocab
tsne = TSNE()
embed_tsne = tsne.fit_transform(embed_mat[:viz_words, :])

fig, ax = plt.subplots(figsize=(16, 16))
for idx in range(viz_words):
    plt.scatter(*embed_tsne[idx, :], color='blue')
    plt.annotate(int_to_vocab[idx], (embed_tsne[idx, 0], embed_tsne[idx, 1]), alpha=1, xytext=(embed_tsne[idx, 0]+1.5, embed_tsne[idx, 1]+1.5), fontsize=12)